# Innholdsfortegnelse
- [Om Guidance](../../../../code/01.Introduce)
- [Oppsett](../../../../code/01.Introduce)
- [Ubegrenset Generering](../../../../code/01.Introduce)
- [Snakke for Phi 3](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [Velg](../../../../code/01.Introduce)
- [Tankerekke](../../../../code/01.Introduce)
- [JSON Generering](../../../../code/01.Introduce)
- [HTML Generering](../../../../code/01.Introduce)


# Om Guidance
Guidance er et velprøvd, åpen kildekode Python-bibliotek for å kontrollere utdata fra enhver språkmodell (LM). Med ett API-kall kan du uttrykke (i Python) de presise programmatiske begrensningene som modellen må følge og generere strukturerte utdata i JSON, Python, HTML, SQL eller hvilken som helst struktur som brukstilfellet krever.

Guidance skiller seg fra konvensjonelle teknikker for prompt. Det håndhever begrensninger ved å styre modellen token for token i inferenslaget, noe som gir høyere kvalitet på utdataene og reduserer kostnader og ventetid med så mye som 30–50 % når det brukes i svært strukturerte scenarier.

For å lære mer om Guidance, besøk [den offentlige GitHub-repositorien](https://github.com/guidance-ai/guidance) eller se [Guidance Breakout Session](https://www.youtube.com/watch?v=qXMNPVVlCMs) på Microsoft Build.


# Oppsett
1. Installer Guidance med `pip install guidance --pre`
2. Distribuer et Phi 3.5 mini-endepunkt i Azure ved å gå til https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml og klikke på "Deploy"-knappen
3. Lagre API-nøkkelen til endepunktet ditt i en miljøvariabel kalt `AZURE_PHI3_KEY` og URL-en i en miljøvariabel kalt `AZURE_PHI3_URL`


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# Ubegrenset generering
Tekst kan genereres uten begrensninger ved å bruke `gen()`-funksjonen. Dette er det samme som å bruke modellen uten Guidance.

## Chat-formattering
Som mange chat-modeller forventer Phi-3 meldinger mellom en bruker og en assistent i et spesifikt format. Guidance støtter Phi-3s chat-mal og vil håndtere chat-formattering for deg. For å lage chat-samtaler, plasser hver del av samtalen i en `with user()` eller `with assistant()` blokk. En `with system()` blokk kan brukes til å sette systemmeldingen.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## Tokenbesparelser
I svært strukturerte scenarier kan Guidance hoppe over tokens og generere kun nødvendige tokens, noe som forbedrer ytelsen, øker effektiviteten og sparer API-kostnader. Genererte tokens vises i denne notatboken med en uthevet bakgrunn. Tvungne tokens vises uten utheving og koster det samme som input-tokens, som er estimert til en tredjedel av kostnaden for output-tokens.

*Merk:* Det første eksempelet med ukontrollert generering kunne ikke tvinge noen tokens fordi vi ikke ga noen begrensninger.


# Snakke for Phi 3
Med Guidance kan du enkelt legge inn tekst i modellens svar. Dette kan være nyttig hvis du ønsker å styre modellens output i en bestemt retning.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# Begrensning med regex
I det forrige eksempelet svarte Phi 3 med oppfølgende forklaringer etter å ha besvart spørsmålet med `Canberra`. For å begrense modellens output til nøyaktig ett ord, kan en regex brukes.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

# Velge mellom flere alternativer
Når noen mulige valg er kjent, kan du bruke funksjonen `select()` for å få modellen til å velge fra en liste med alternativer.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

Med `select()`, ble bare tokenen `Kan` generert. Fordi `Canberra` er det eneste alternativet som muligens kan fullføre svaret, ble de gjenværende tokenene tvunget.


# Tankerekke
Tankerekke er en teknikk som kan bidra til å forbedre kvaliteten på modellens output ved å oppmuntre den til å løse et problem steg for steg. Vanligvis kreves det flere prompt-runder for å komme frem til et endelig svar. Først instrueres modellen til å tenke steg for steg. Deretter promptes modellen igjen for å gi det endelige svaret. Med standard chat-inferens-API-er krever dette 2 API-kall, og modellens genererte "tankerekke" blir belastet to ganger – én gang som output-tokens når modellen genererte den, og deretter igjen som input-tokens for det andre kallet. Med Guidance blir hele den flertrinns prosessen behandlet og belastet som en del av ett enkelt API-kall, noe som reduserer kostnader og ventetid.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# Generering av JSON
Guidance kan brukes for å sikre generering av JSON som er i samsvar med et JSON-skjema eller en pydantic-modell, slik som brukerskjemaet vist her.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## HTML-generering

Guidance kan også brukes til å generere kode og følge de syntaktiske kravene i programmeringsspråket. I denne delen skal vi lage et lite Guidance-program for å skrive veldig enkle HTML-nettsider.

Vi deler opp nettsiden i mindre seksjoner, hver med sin egen Guidance-funksjon. Disse kombineres deretter i vår endelige funksjon for å lage en HTML-nettside. 
Deretter kjører vi denne funksjonen mot en Guidance-aktivert modell i Azure AI.

*Merk:* Dette kommer ikke til å være en fullverdig HTML-generator; målet er å vise hvordan du kan lage strukturert output for dine individuelle behov.

Vi begynner med å importere det vi trenger fra Guidance:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML-nettsider er svært strukturerte, og vi vil 'tvinge' disse delene av siden ved hjelp av Guidance.  
Når vi eksplisitt ber om tekst fra modellen, må vi sørge for at den ikke inneholder noe som kan være en tagg - det vil si, vi må utelukke tegnene '<' og '>'.  


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

Vi kan deretter bruke denne funksjonen til å generere tekst innenfor en vilkårlig HTML-tag:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

Nå, la oss lage sideoverskriften.  
Som en del av dette, må vi generere en sidetittel:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

Kroppen av HTML-siden kommer til å fylles med overskrifter og avsnitt.  
Vi kan definere en funksjon for å gjøre hver av dem:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

Nå, funksjonen for å definere selve kroppen av HTML-en.  
Denne bruker `select()` med `recurse=True` for å generere flere overskrifter og avsnitt:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

Neste kommer vi til funksjonen som genererer den komplette HTML-siden.  
Vi legger til HTML-starttaggen, deretter genererer vi headeren, deretter kroppen, og til slutt legger vi til den avsluttende HTML-taggen:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

Vi tilbyr en brukervennlig innpakning, som lar oss:
- Angi temperaturen for genereringen
- Fange den genererte siden fra Model-objektet


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

Vi kan deretter skrive outputen til en fil:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

Og [se resultatet](../../../../code/01.Introduce/sample_page.html).



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
